### Imports needed

In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

from sklearn.tree import export_graphviz
import pydotplus

from sklearn.metrics import confusion_matrix

import numpy as np

## Load Iris data

In [2]:
iris = datasets.load_iris()
X = iris.data
Y = iris.target
features = iris.feature_names

In [3]:
'''
    Store feature names and its corresponding indices initially. Otherwise in every recusive call we need to make new X ndarray
    which increases complexity.
'''
feature_indices = {}
for i in range(len(features)):
    feature_indices[features[i]] = feature_indices.get(features[i],0) + i

### Functions used

In [4]:
def entropy_node(Y):
    total_freq = len(Y)

    # store different classes possible with its frequency
    class_freq = {}
    for c in Y:
        class_freq[c] = class_freq.get(c,0) + 1
        
    # Now we calculate entropy or info required
    info_req =  0
    for k in class_freq:
        prob_k = class_freq[k]/total_freq
        if prob_k!=0:
            info_req += (-1 * prob_k * np.log2(prob_k))
    return info_req

In [13]:
# calculate entropy(info required) after spliting on basis of feature having index f_i

def info_fi(X, Y, f_i, boundry):
    entropy_after_split = np.float(0)
    # feature on which we goona split
    feature = X[:, f_i] # feature corresponds to index f_i
    Y_left = Y[np.where(feature <= boundry)]
    Y_right = Y[np.where(feature > boundry)]
    
    if len(Y_left)!=0:
        entropy_after_split += ( np.float(len(Y_left)/len(Y)) * entropy_node(Y_left) ) 
    if len(Y_right) !=0 :
        entropy_after_split += ( np.float(len(Y_right)/len(Y)) * entropy_node(Y_right) )
            
    return entropy_after_split

In [14]:
# calculate split_info of feature having index f_i

def find_split_info(X, Y, f_i, boundry):
    size = len(Y)
    split_info = np.float(0)
    
    # feature on which we goona split
    feature = X[:, f_i] # feature corresponds to index f_i
    Y_left = Y[np.where(feature <= boundry)]
    Y_right = Y[np.where(feature > boundry)]
    
    if len(Y_left) != 0:
        split_info += np.float( -1*np.float(len(Y_left)/size) * np.log2(len(Y_left)/size) ) 
    if len(Y_right) != 0:
        split_info += np.float( -1*np.float(len(Y_right)/size) * np.log2(len(Y_right)/size) )
        
    return split_info

In [15]:
def DecisionTree(X, Y, features, level=0):
    # Print some info as required
    print("Level",level) 
    
    # store different classes possible with its frequency
    class_freq = {}
    for c in Y:
        class_freq[c] = class_freq.get(c,0) + 1
    
    for c in class_freq:
        if class_freq[c] != 0:
            print("Count of",c,"=",class_freq[c])
    
    # Find current entropy
    entropy_current = entropy_node(Y)
    
    
    # Base case
    # If node is pure, 
    if len(set(Y))==1 :
        print("Current Entropy is = 0.0")
        print("Reached Leaf Node")
    
    # If no feature left to split
    elif len(features) == 0:
        print("Current Entropy is =",entropy_current)
        print("Reached Leaf Node")
    
    else:
        print("Current Entropy is =",entropy_current)
        
        # Find max info_gain
        max_gain_ratio = 0
        max_gain_ratio_feature = features[0]
        X_feature_boundry = -1 # initially, we change this inside loop
        
        # check info_gain for each feature
        for f in features:
            '''
                Since in Iris dataset all features have continuous data. So we need to find a boundry
                To find an boundry we first sort feature data, and then try boundry as middle value b/w 2 consecutive data
            '''
            X_feature = X[:, feature_indices[f]]
            X_feature.sort()
            #X_feature_boundry = X_feature[0]/2 # Assume for now, It may be changed below
            #max_gain_ratio = info_fi(X, Y, feature_indices[f], X_feature_boundry) / find_split_info(X, Y, feature_indices[f], X_feature_boundry)
            
            # Try different boundries
            X_len = len(X_feature)
            if X_len == 1:
                print("len is 1")
                continue
                
            for i in range(X_len-1):
                curr_boundry = np.float(X_feature[i]+X_feature[i+1])/2
                info_after_split_f = info_fi(X, Y, feature_indices[f], curr_boundry)
                info_gain_f = entropy_current - info_after_split_f
                split_info_f = find_split_info(X, Y, feature_indices[f], curr_boundry)
                gain_ratio_f = np.float(info_gain_f/split_info_f)
            
                if gain_ratio_f>max_gain_ratio:
                    max_gain_ratio = gain_ratio_f
                    max_gain_ratio_feature = f
                    X_feature_boundry = curr_boundry
        
        print("Splitting on feature",max_gain_ratio_feature, "<=",X_feature_boundry, "with gain ratio", max_gain_ratio)
        
        # Now split on basis of max_gain_ratio_feature and X_feature_boundry and call on two sides of boundry
        # 1st remove this max_gain_ratio_feature from current features list
        new_features = []
        for f in features:
            if f != max_gain_ratio_feature:
                new_features.append(f)
                
        f_i = feature_indices[max_gain_ratio_feature] # bcz this index(f_i) gives us the exact column correspond to required featue 
        f_i_feature = X[:, f_i] # feature corresponds to index f_i
        
        # Recursive calls on 2 sides of boundry
        X_left = X[np.where(f_i_feature <= X_feature_boundry)]
        Y_left = Y[np.where(f_i_feature <= X_feature_boundry)]
        print()
        DecisionTree(X_left, Y_left, new_features, level+1)
        
        X_right = X[np.where(f_i_feature > X_feature_boundry)]
        Y_right = Y[np.where(f_i_feature > X_feature_boundry)]
        print()
        DecisionTree(X_right, Y_right, new_features, level+1)
        
        

### Call Decision Tree

In [16]:
DecisionTree(X,Y,features)

Level 0
Count of 0 = 50
Count of 1 = 50
Count of 2 = 50
Current Entropy is = 1.584962500721156
Splitting on feature petal length (cm) <= 1.9 with gain ratio 0.9999999999999999

Level 1
Count of 0 = 50
Current Entropy is = 0.0
Reached Leaf Node

Level 1
Count of 1 = 50
Count of 2 = 50
Current Entropy is = 1.0
Splitting on feature sepal length (cm) <= 6.2 with gain ratio 0.929259315921287

Level 2
Count of 1 = 49
Current Entropy is = 0.0
Reached Leaf Node

Level 2
Count of 1 = 1
Count of 2 = 50
Current Entropy is = 0.13923299905509887
Splitting on feature petal width (cm) <= 1.6 with gain ratio 0.26402404255823636

Level 3
Count of 1 = 1
Count of 2 = 2
Current Entropy is = 0.9182958340544896
Splitting on feature sepal width (cm) <= -1 with gain ratio 0

Level 4
Current Entropy is = 0
Reached Leaf Node

Level 4
Count of 1 = 1
Count of 2 = 2
Current Entropy is = 0.9182958340544896
Reached Leaf Node

Level 3
Count of 2 = 48
Current Entropy is = 0.0
Reached Leaf Node


c:\users\mehak\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:59: RuntimeWarning: invalid value encountered in double_scalars


In [12]:
np.float(0)

0.0